In [1]:
import pandas as pd
import numpy as np

In [2]:
questions = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/VLAT Questions.csv")
questions.head()

,id,dropped,vis,item,question:,option:,correct
0,1,no,VLAT_a,a_1,What was the price of a barrel of oil in Febru...,$57.36; $47.82; $50.24; $39.72,$50.24
1,2,no,VLAT_a,a_2,In which month was the price of a barrel of oi...,March; May; July; December,December
2,3,no,VLAT_a,a_3,What was the price range of a barrel of oil in...,$35 - $65; $48.36 - $60.95; $37.04 - $48.36; $...,$37.04 - $60.95
3,4,no,VLAT_a,a_4,"Over the course of the second half of 2015, th...",rising; falling; staying,falling
4,5,no,VLAT_a,a_5,About how much did the price of a barrel of oi...,$4; $15; $17; $45,$15


In [3]:
dropped_questions = set(questions[questions['dropped'] == "yes"]['id'].to_numpy())
dropped_questions

{13, 24, 26, 30, 39, 43, 50, 58}

In [4]:
questions['num_options'] = questions['option:'].apply(lambda x: len(x.split(";")))
questions.tail(3)

,id,dropped,vis,item,question:,option:,correct,num_options
58,59,no,VLAT_l,l_2,For which website was the number of unique vis...,Facebook; Amazon; Bing; Google,Google,4
59,60,no,VLAT_l,l_3,The number of unique visitors for Amazon was m...,True; False,FALSE,2
60,61,no,VLAT_l,l_4,Samsung is nested in the Financial category.,True; False,FALSE,2


In [5]:
questions = questions.set_index("id")

In [6]:
questions_meta = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/VLAT Questions Metadata.csv")
questions_meta = questions_meta.set_index("id").fillna("")
questions_meta.tail(3)

,vis,task,subtask,difficulty,discrimination
id,,,,,
59,treemap,find extremum,relative value,moderate,high
60,treemap,make comparisons,relative value,hard,high
61,treemap,identify hierarchical structure,,easy,medium


In [7]:
results_1 = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/Results/3_GEMNI 2.0 Pro_Final Results with Explanation/Gemini_VLAT_1741123345.csv")
results_2 = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/Results/3_GEMNI 2.0 Pro_Final Results with Explanation/Gemini_VLAT_1741134983.csv")
results_3 = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/Results/3_GEMNI 2.0 Pro_Final Results with Explanation/Gemini_VLAT_1741136868.csv")

In [8]:
results = results_1.merge(results_2, on='id', suffixes=("", "_2"))
results = results.merge(results_3, on='id', suffixes=("", "_3"))
results = results[~results['id'].isin(dropped_questions)]
results = results.set_index("id")

In [9]:
len(results)

53

In [10]:
results.head()

,response,time,correct_bool,response_2,time_2,correct_bool_2,response_3,time_3,correct_bool_3
id,,,,,,,,,
1,50.24,7.469351,True,50.24,6.888034,True,50.24,7.419151,True
2,December,6.981137,True,December,6.162074,True,December,6.644902,True
3,$37.04 - $60.95,7.428970,True,$37.04 - $60.95,8.159966,True,$37.04 - $60.95,6.938839,True
4,falling,6.400615,True,falling,6.870172,True,falling,7.609844,True
5,15,5.465111,True,$15,5.011563,True,$15,5.989933,True


In [11]:
responses = results[['response', 'response_2', 'response_3']]
responses.head()

,response,response_2,response_3
id,,,
1,50.24,50.24,50.24
2,December,December,December
3,$37.04 - $60.95,$37.04 - $60.95,$37.04 - $60.95
4,falling,falling,falling
5,15,$15,$15


In [12]:
agreements = np.logical_and(responses['response'] == responses['response_2'], responses['response_2'] == responses['response_3'])
responses[~agreements]

,response,response_2,response_3
id,,,
5,15,$15,$15
10,9,9,$9
17,Some College Degree,College Graduate,College Graduate
22,240,240,390
27,54,54,53.9 kg
28,197.1 cm,197.1 cm,196 cm
29,53.9 - 123.6 kg,40 - 130 kg,53.9 - 123.6 kg
36,14-Jun,14-Dec,14-Dec
37,$4.6 - $6.0,$4.6 - $5.9,$4.6 - $6.0


In [13]:
# Correct way to combine multiple logical conditions
omissions = np.logical_or(
    np.logical_or(responses['response'] == "Omit", responses['response_2'] == "Omit"),
    responses['response_3'] == "Omit"
)

# Then filter
responses[omissions]

,response,response_2,response_3
id,,,


In [14]:
scoring = results.copy()
scoring["correct_bool"] = np.where(scoring["response"] == "Omit", "Omit", scoring["correct_bool"])
scoring["correct_bool_2"] = np.where(scoring["response_2"] == "Omit", "Omit", scoring["correct_bool_2"])
scoring["correct_bool_3"] = np.where(scoring["response_3"] == "Omit", "Omit", scoring["correct_bool_3"])
scoring.head(6)

,response,time,correct_bool,response_2,time_2,correct_bool_2,response_3,time_3,correct_bool_3
id,,,,,,,,,
1,50.24,7.469351,True,50.24,6.888034,True,50.24,7.419151,True
2,December,6.981137,True,December,6.162074,True,December,6.644902,True
3,$37.04 - $60.95,7.428970,True,$37.04 - $60.95,8.159966,True,$37.04 - $60.95,6.938839,True
4,falling,6.400615,True,falling,6.870172,True,falling,7.609844,True
5,15,5.465111,True,$15,5.011563,True,$15,5.989933,True
6,15 Mbps,7.732076,True,15 Mbps,6.809267,True,15 Mbps,6.954838,True


In [15]:
def getScore(correct_arr):
    score = 0
    for i, item in enumerate(correct_arr):
        if item == "True":
            score += 1
        elif item == "False":
            score -= 1/(questions.loc[i,"num_options"] - 1)
    return score

In [16]:
for col in ["correct_bool", "correct_bool_2", "correct_bool_3"]:
    score = sum(scoring[col] == "True")
    print(col+":", round(score, 2), "(raw)")

correct_bool: 40 (raw)
correct_bool_2: 40 (raw)
correct_bool_3: 40 (raw)


In [17]:
for col in ["correct_bool", "correct_bool_2", "correct_bool_3"]:
    score = getScore(scoring[col].to_numpy())
    print(col+":", round(score, 2))

correct_bool: 34.17
correct_bool_2: 33.5
correct_bool_3: 34.17


In [18]:
# Chain logical_and operations correctly
scoring["correct_bool_consensus"] = np.where(
    # First logical_and chain for the condition
    np.logical_and(
        np.logical_and(scoring["response"] == "Omit", scoring["response_2"] == "Omit"),
        scoring["response_3"] == "Omit"
    ), 
    "Omit",
    # Second logical_and chain for the value
    np.logical_and(
        np.logical_and(scoring["correct_bool"] == "True", scoring["correct_bool_2"] == "True"),
        scoring["correct_bool_3"] == "True"
    )
)

# View the results
scoring[["correct_bool", "correct_bool_2", "correct_bool_3", "correct_bool_consensus"]].head(10)

,correct_bool,correct_bool_2,correct_bool_3,correct_bool_consensus
id,,,,
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
5,True,True,True,True
6,True,True,True,True
7,True,True,True,True
8,True,True,True,True
9,False,False,False,False


In [19]:
# First convert string "True" to actual boolean values if needed
scoring["correct_bool_consensus"] = scoring["correct_bool_consensus"].astype(str) == "True"

# Now calculate the raw score (sum of True values)
consensus_score_raw = scoring["correct_bool_consensus"].sum()
print(round(consensus_score_raw, 2), "(raw)")

# If getScore is a custom function, make sure it works with boolean arrays
# If it's not defined, you might need to implement it
consensus_score = getScore(scoring["correct_bool_consensus"].to_numpy())
print(round(consensus_score, 2))

35 (raw)
0


In [20]:
correct_bool_consensus = scoring[["correct_bool_consensus"]]
questions_meta_concensus_ans = correct_bool_consensus.join(questions_meta)
questions_meta_concensus_ans.head(5)

,correct_bool_consensus,vis,task,subtask,difficulty,discrimination
id,,,,,,
1,True,line chart,retrieve value,,easy,low
2,True,line chart,find extremum,,easy,low
3,True,line chart,determine range,,moderate,high
4,True,line chart,find correlations/trends,,easy,low
5,True,line chart,make comparisons,,moderate,high


In [21]:
for col in ['vis', 'task', 'difficulty']:
    vals = questions_meta_concensus_ans[questions_meta_concensus_ans['correct_bool_consensus'] == "True"][col].value_counts()
    print(vals, "\n")

Series([], Name: vis, dtype: int64) 

Series([], Name: task, dtype: int64) 

Series([], Name: difficulty, dtype: int64) 



In [22]:
for col in ['vis', 'task', 'difficulty']:
    vals = questions_meta_concensus_ans[questions_meta_concensus_ans['correct_bool_consensus'] == "False"][col].value_counts()
    print(vals, "\n")

Series([], Name: vis, dtype: int64) 

Series([], Name: task, dtype: int64) 

Series([], Name: difficulty, dtype: int64) 



In [23]:
for col in ['vis', 'task', 'difficulty']:
    vals = questions_meta_concensus_ans[questions_meta_concensus_ans['correct_bool_consensus'] == "Omit"][col].value_counts()
    print(vals, "\n")

Series([], Name: vis, dtype: int64) 

Series([], Name: task, dtype: int64) 

Series([], Name: difficulty, dtype: int64) 



In [24]:
questions_meta_concensus_ans[["vis", "correct_bool_consensus"]].value_counts(sort=False)

vis                     correct_bool_consensus
100% stacked bar chart  False                     1
                        True                      2
area chart              False                     2
                        True                      2
bar chart               False                     1
                        True                      3
bubble chart            False                     1
                        True                      6
choropleth map          False                     2
                        True                      1
histogram               False                     2
                        True                      1
line chart              True                      5
pie chart               True                      3
scatterplot             False                     3
                        True                      4
stacked area chart      False                     3
                        True                      3
stacked bar chart

In [25]:
questions_meta_concensus_ans[["task", "correct_bool_consensus"]].value_counts(sort=False)

task                             correct_bool_consensus
determine range                  False                     2
                                 True                      3
find anomalies                   True                      2
find clusters                    False                     1
                                 True                      1
find correlations/trends         True                      5
find extremum                    False                     5
                                 True                      7
identify hierarchical structure  True                      1
make comparisons                 False                     4
                                 True                      9
retrieve value                   False                     6
                                 True                      7
dtype: int64

In [26]:
questions_meta_concensus_ans[["difficulty", "correct_bool_consensus"]].value_counts(sort=False)

difficulty  correct_bool_consensus
easy        False                      3
            True                      14
hard        False                      8
            True                       9
moderate    False                      7
            True                      12
dtype: int64

In [27]:
# Correct, hard
questions_meta_concensus_ans[np.logical_and(questions_meta_concensus_ans['correct_bool_consensus'] == "True",
                                            questions_meta_concensus_ans['difficulty'] == "hard")]

,correct_bool_consensus,vis,task,subtask,difficulty,discrimination
id,,,,,,
